In [1]:
import tensorflow as tf
import onnxruntime as rt
import tf2onnx
import os
import pandas as pd
from time import time
import numpy as np
os.chdir('..')
from MakeModelClassifier import MakeModelClassifier as mm

Num GPUs Available:  0


## Instantiate MakeModelClassifier with 50% sample of Stanford car data

In [2]:
config = {'img_df': '/Users/josephking/Documents/sponsored_projects/MERGEN/data/vehicle_classifier/stanford_car_data/Bboxes.csv',
          'data': '/Users/josephking/Documents/sponsored_projects/MERGEN/data/vehicle_classifier/stanford_car_data',
          'output': '/Users/josephking/Documents/sponsored_projects/MERGEN/scripts/output',
          'logging': False,
          'train': False,
          'predict': True,
          'weights': '/Users/josephking/Documents/sponsored_projects/MERGEN/output/MakeModelClassifier/2021-11-17-00h53/training_checkpoints',
          'min_class_img_count': 0,
          'pixel_dilation': 0,
          'sample' : 0.5,
          'seed': 123,
          'img_size': (224, 224),
          'batch_size': 32,
          'model': 'resnet',
          'resnet_size': '50',
          'dropout': 0.2,
          'units2': 4096,
          'units1': 2048,
          'optimizer': 'adam',
          'learning_rate': 0.0001
}

In [3]:
mmc = mm(config)

In [4]:
_, _, test = mmc.image_pipeline(predict=True)


Reading in and processing images.



In [5]:
print(test)

<PrefetchDataset shapes: ((None, 224, 224, 3), (None, 574)), types: (tf.float32, tf.int64)>


In [6]:
print(f'\nNumber of test images: {len(mmc.df)}')


Number of test images: 2600


## Predictions with Tensorflow Keras model

In [7]:
latest = tf.train.latest_checkpoint(config['weights'])
keras_model = mmc.build_compile_model()
keras_model.load_weights(latest)

In [8]:
start = time()
keras_predictions = keras_model.predict(test)
print("\nTotal prediction time in seconds: {:.2f}\n".format((time()-start)))


Total prediction time in seconds: 95.21



## Convert Keras weights to Onnx

In [9]:
spec = (tf.TensorSpec(((None,) + mmc.config['img_size'] + (3,)), tf.float32, name="input"),)
onnx_path = os.path.join(mmc.config['weights'], 'model.onnx')

start = time()
model_proto, _ = tf2onnx.convert.from_keras(keras_model, input_signature=spec, opset=13, output_path=onnx_path)
output_names = [n.name for n in model_proto.graph.output]
print("\nTotal time spent converting weights in seconds: {:.2f}\n".format((time()-start)))

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`

Total time spent converting weights in seconds: 31.12



In [10]:
print(output_names)

['dense_2']


## Predictions with Onnx weights

#### Convert to Numpy array

In [11]:
converted_test = list(test.as_numpy_iterator())

In [12]:
array = np.array(converted_test[0][0])
for i in range(1, len(converted_test)):
    array = np.concatenate((array, converted_test[i][0]), axis=0)

In [13]:
onnx_model = rt.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])  # run on a MacBook with no GPU

start = time()
onnx_pred = onnx_model.run(output_names, {"input": array})
print("\nTotal prediction time in seconds: {:.2f}\n".format((time()-start)))


Total prediction time in seconds: 745.53



##  Compare softmax probabilities

In [14]:
assert(keras_predictions.shape == onnx_pred[0].shape)

In [15]:
np.testing.assert_allclose(keras_predictions, onnx_pred[0], rtol=1e-5)

AssertionError: 
Not equal to tolerance rtol=1e-05, atol=0

Mismatched elements: 514292 / 1492400 (34.5%)
Max absolute difference: 6.6310167e-06
Max relative difference: 1.
 x: array([[1.440194e-13, 9.826511e-11, 6.791989e-13, ..., 4.365169e-09,
        2.501970e-07, 1.148975e-11],
       [1.994335e-10, 1.379016e-10, 2.057311e-12, ..., 3.600836e-09,...
 y: array([[1.440177e-13, 9.826344e-11, 6.791848e-13, ..., 4.365178e-09,
        2.501956e-07, 1.148958e-11],
       [1.994339e-10, 1.379008e-10, 2.057353e-12, ..., 3.600842e-09,...

##  Compare argmax(0) between models

#### True labels

In [16]:
images, labels = tuple(zip(*test))  # Recover labels

label_df = pd.DataFrame()
for x in range(len(labels)):
    label_df = pd.concat([label_df, pd.DataFrame(labels[x].numpy())], axis=0)
label_df = label_df.reset_index(drop=True)
label_series = label_df.idxmax(axis=1).astype(str)
label_series.replace(to_replace=mmc.label_mapping, inplace=True)

In [17]:
label_series

0       Hyundai Veracruz
1                 BMW X6
2          Dodge Charger
3           BMW 3 Series
4          Dodge Caliber
              ...       
2595      Hyundai Sonata
2596       Buick Enclave
2597      Suzuki Kizashi
2598       Buick Enclave
2599              BMW M3
Length: 2600, dtype: object

#### Predicted class

In [18]:
colnames = []
for key in mmc.label_mapping.keys():
    colnames.append(mmc.label_mapping[key])
keras_pred_df = pd.DataFrame(keras_predictions, columns=colnames)
onnx_pred_df = pd.DataFrame(onnx_pred[0], columns=colnames)

In [19]:
(keras_pred_df.idxmax(axis=1).astype(str) == label_series).mean()

0.6088461538461538

In [20]:
(onnx_pred_df.idxmax(axis=1).astype(str) == label_series).mean()

0.6088461538461538

In [21]:
assert ((keras_pred_df.idxmax(axis=1).astype(str) == onnx_pred_df.idxmax(axis=1).astype(str)).all())